# Wikipedia API Scrapper

In dieser Sektion greifen wir auf Wikipedia via die wikipediaapi Library zu. Dies ist ein bekannter Ansatz, um Inforamtionen von Wikipedia zu bekommen. Unsere Funktion greift auf das jeweilige Topic zu und schliesst alle sub-pages ein. Damit die Textlänge nicht ausufert, nimmt der Scrapper jeweils nur den ersten Abschnitt, also das Abstract, eines Themas und setzt dieses mit den anderen zusammen. So ergibt sich ein breiter Text mit vielen Subthemen des ausgewählten Themas. 

### Requiremend

In [1]:
# !pip3 install wikipedia-api
import wikipediaapi
import pandas as pd
import sys
sys.path.append(r"..")
from nlp_functions import (classifier, remove_colons, remove_digits, remove_n,
                           remove_redundant_whitespaces,
                           remove_strange_characters, remove_stripes,
                           text_loader)

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\daveb\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
# Define language and page
wiki = wikipediaapi.Wikipedia('en')


In [3]:
# Get members for a category together with a short description
def members_collector_summary(category):
      mdict = {}
      categorymembers = category.categorymembers
      for c in categorymembers.values():
        if c.ns == 0:                                                            # Exclude categories within the category
          mdict[c.title] = c.summary
      return mdict


In [4]:
# Get members for a category together with a short description
def members_collector_fulltext(category):
      mdict = {}
      categorymembers = category.categorymembers
      for c in categorymembers.values():
        if c.ns == 0:                                                            # Exclude categories within the category
          mdict[c.title] = c.text
      return mdict


In [5]:
def scrape_to_text_fulltext(topic):
    import pandas as pd
    import wikipediaapi

    # Define language and page
    wiki = wikipediaapi.Wikipedia('en')
    # Setting the sustainability Category
    wiki_topic = wiki.page(f"Category:{topic}")
    topic_dict = members_collector_fulltext(wiki_topic) 

    # Create the dataframe for sustainability
    df_topic = pd.DataFrame([topic_dict.keys(), topic_dict.values()]).T
    df_topic.columns = ['Article', 'Description']
    df_topic
    topic_text = []
    topic_list = df_topic["Description"][:]
    for i in topic_list:
            topic_text.append(i)
    return topic_text

In [6]:
def scrape_to_text_summary(topic):
    import pandas as pd
    import wikipediaapi

    # Define language and page
    wiki = wikipediaapi.Wikipedia('en')
    # Setting the sustainability Category
    wiki_topic = wiki.page(f"Category:{topic}")
    topic_dict = members_collector_summary(wiki_topic)

    # Create the dataframe for sustainability
    df_topic = pd.DataFrame([topic_dict.keys(), topic_dict.values()]).T
    df_topic.columns = ['Article', 'Description']
    df_topic
    topic_text = []
    topic_list = df_topic["Description"][:]
    for i in topic_list:
            topic_text.append(i)
    return topic_text

In [7]:
label_candidate = ['sustainability', 'human rights', 'fraud', 'social issues', 'labour law'] # no data with employee affairs
label_fulltext_list = []
wiki_dict = {"Label": [], "Text": []}

for i in label_candidate:
    label_fulltext_list.append(scrape_to_text_fulltext(i))

In [8]:
label_candidate = ['sustainability', 'human rights', 'fraud', 'social issues', 'labour law'] # no data with employee affairs
label_summary_list = []
wiki_dict_fulltext = {"Label": [], "Text": []}

for i in label_candidate:
    label_summary_list.append(scrape_to_text_summary(i))

In [9]:
wiki_dict_fulltext = dict(zip(label_candidate, label_fulltext_list))

In [10]:
wiki_dict_summary = dict(zip(label_candidate, label_summary_list))

In [11]:
for k, v in wiki_dict_fulltext.items():
    wiki_dict_fulltext[k] = " ".join(v)

    wiki_dict_fulltext[k] = remove_strange_characters(v)

In [12]:
for k, v in wiki_dict_fulltext.items():
    wiki_dict_fulltext[k] = remove_n(v)

    wiki_dict_fulltext[k] = remove_colons(v)

    wiki_dict_fulltext[k] = v.replace(r" .",".")
    wiki_dict_fulltext[k] = v.replace(r"..",".")
    wiki_dict_fulltext[k] = v.replace(r"...",".")

    wiki_dict_fulltext[k] = remove_stripes(v)

    wiki_dict_fulltext[k] = remove_redundant_whitespaces(v)
    wiki_dict_fulltext[k] = remove_digits(v)
    print(type(k), type(v))

<class 'str'> <class 'str'>
<class 'str'> <class 'str'>
<class 'str'> <class 'str'>
<class 'str'> <class 'str'>
<class 'str'> <class 'str'>


In [13]:
for k, v in wiki_dict_summary.items():
    wiki_dict_summary[k] = " ".join(v)

    wiki_dict_summary[k] = remove_strange_characters(v)

In [14]:
for k, v in wiki_dict_summary.items():
    wiki_dict_summary[k] = remove_n(v)

    wiki_dict_summary[k] = remove_colons(v)

    wiki_dict_summary[k] = v.replace(r" .",".")
    wiki_dict_summary[k] = v.replace(r"..",".")
    wiki_dict_summary[k] = v.replace(r"...",".")

    wiki_dict_summary[k] = remove_stripes(v)

    wiki_dict_summary[k] = remove_redundant_whitespaces(v)
    wiki_dict_summary[k] = remove_digits(v)
    print(type(k), type(v))

<class 'str'> <class 'str'>
<class 'str'> <class 'str'>
<class 'str'> <class 'str'>
<class 'str'> <class 'str'>
<class 'str'> <class 'str'>


In [15]:
df = pd.DataFrame(list(wiki_dict_fulltext.items()),columns = ['Label','Text']) 
df.to_csv(r"..\Data\Resultate\Testfolder\wiki_artikel_fulltext.csv")

In [16]:
df = pd.DataFrame(list(wiki_dict_summary.items()),columns = ['Label','Text']) 
df.to_csv(r"..\Data\Resultate\Testfolder\wiki_artikel_summary.csv")